In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

In [2]:
df = pd.read_csv('../constants_and_dicts/state_fips.csv')
state_to_fips = dict(zip(df.Postal,df.FIPS))
# state_to_fips

In [15]:
# problem dfs:
# XX_blockgroup_acs5_2021_healthinsurance.csv
# XX_blockgroup_acs5_2021_income.csv


def normalize_attrs_by_pop(df, f):
#     print(f)
    f = (f[f.rindex('/')+1:-4])
    f = f[f.rindex('_')+1:]
    if f=='healthinsurance':
        df['HINS_A0018'] = df['HINS_A0018'].div(df['POP_A0018'])
        df['HINS_A1934'] = df['HINS_A1934'].div(df['POP_A1934'])
        df['HINS_A3564'] = df['HINS_A3564'].div(df['POP_A3564'])
        df['HINS_A65p'] = df['HINS_A65p'].div(df['POP_A65p'])
#         display(df.head())
    elif f=='income':
#         print(f)
#         display(df.head())
        df.set_index('GEOID',drop=True,inplace=True)
        denom = df['HH']
        df = df.apply(lambda x: x/denom, axis=0)        
#         display(df.head())
    else:
        df.set_index('GEOID',drop=True,inplace=True)
        universe = df.columns[0]
#         print(col_name)
        denom = df[universe]
        df = df.apply(lambda x: x/denom, axis=0)
    df = df.reset_index(inplace=False, drop=False)
    return df

# def get_state_df(files):
#     first_df = None
#     for i,f in enumerate(files):
#         this_df = pd.read_csv(f)
#         display(this_df)
#         if first_df is None:
#             first_df = this_df
#         else:
#             df = pd.merge(first_df, this_df, on='GEOID', suffixes=(f'_x{i}', f'_x{i+1}'))
#             first_df = df
#     return df


def get_state_df(files):
    first_df = None
    for i,f in enumerate(files):
        this_df = pd.read_csv(f)
        this_df = normalize_attrs_by_pop(this_df, f)
        if first_df is None:
            first_df = this_df
        else:
            df = pd.merge(first_df, this_df, on='GEOID', suffixes=(f'_x{i}', f'_x{i+1}'))
            first_df = df
    return df

In [20]:
import glob
import os

acs_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/raw/acs_results/'

all_st_dfs = []

state_dirs = [x for x in os.walk(acs_dir)][0][1]

for i,state in enumerate(state_dirs):
    if i==0:
        continue    
        
    
    files = glob.glob(f'{acs_dir}{state}/*.csv')
    
    df = get_state_df(files)
    
#     first_df = None
#     for i,f in enumerate(files):
#         this_df = pd.read_csv(f)
#         this_df = normalize_attrs_by_pop(this_df, f)
        
#         if first_df is None:
#             first_df = this_df
#         else:
#             df = pd.merge(first_df, this_df, on='GEOID', suffixes=(f'_x{i}', f'_x{i+1}'))
#             first_df = df
    fips = state_to_fips[state]
    df['State_fips'] = fips
    df['State'] = state
    df = df[['State'] + ['State_fips'] + 
            [col for col in df.columns if col not in ['State_fips','State']]]

    all_st_dfs.append(df)
#     break
    
#     if i==10:
#         break
    
final_df = pd.concat(all_st_dfs)
final_df.rename({'GEOID':'Fips'},axis=1,inplace=True)
final_df.reset_index(inplace=True,drop=True)
# final_df  
                        
    # # # # # #

In [21]:
final_df

,State,State_fips,Fips,POP_x2,POP_M,POP_F,POP_A0004,POP_A0509,POP_A1014,POP_A1517,...,HU_x15,HU_UIS01D,HU_UIS01A,HU_UIS02,HU_UIS0304,HU_UIS0509,HU_UIS1019,HU_UIS2049,HU_UIS50P,HU_UISOTHER
0,AL,1,1001,1.0,0.484315,0.515685,0.056972,0.063806,0.068751,0.047442,...,1.0,0.731940,0.012371,0.002193,0.016549,0.022880,0.016384,0.011874,0.012040,0.173769
1,AL,1,1003,1.0,0.487741,0.512259,0.052987,0.051644,0.072236,0.038982,...,1.0,0.643882,0.012910,0.016023,0.019571,0.032038,0.035906,0.020622,0.115585,0.103463
2,AL,1,1005,1.0,0.528960,0.471040,0.052259,0.063383,0.056297,0.036462,...,1.0,0.546927,0.009943,0.067284,0.042256,0.019542,0.004628,0.003600,0.004800,0.301020
3,AL,1,1007,1.0,0.548813,0.451187,0.053364,0.056845,0.054881,0.040692,...,1.0,0.614224,0.003218,0.006768,0.002996,0.025630,0.006324,0.002108,0.009542,0.329191
4,AL,1,1009,1.0,0.501494,0.498506,0.058878,0.060764,0.071225,0.041505,...,1.0,0.690341,0.009622,0.016716,0.015412,0.010437,0.002976,0.005382,0.000041,0.249072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,WY,56,56037,1.0,0.514614,0.485386,0.062578,0.081396,0.070845,0.042959,...,1.0,0.596954,0.044174,0.024930,0.053353,0.029154,0.026807,0.021070,0.003807,0.199750
3108,WY,56,56039,1.0,0.526609,0.473391,0.045800,0.044256,0.063253,0.024658,...,1.0,0.587552,0.099132,0.041871,0.082837,0.079668,0.038099,0.030479,0.007318,0.033044
3109,WY,56,56041,1.0,0.508774,0.491226,0.068880,0.083455,0.085893,0.048065,...,1.0,0.593038,0.068715,0.011226,0.026874,0.026080,0.048078,0.014174,0.007030,0.204785
3110,WY,56,56043,1.0,0.515448,0.484552,0.051365,0.041323,0.078012,0.056900,...,1.0,0.741020,0.022644,0.033316,0.069495,0.033316,0.005726,0.004425,0.001301,0.088756


In [23]:
final_df.to_csv(f'{DATA_DIR}/processed/all_county_acs_normed_covs_{DT}.csv',index=False)